# SetUp

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

# Get Data

In [2]:
df = pd.read_csv('../data/bank-additional/bank-additional-full.csv', sep = ';')

In [3]:
df.size

864948

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [5]:
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [7]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [8]:
df.groupby('poutcome')['poutcome'].count()

poutcome
failure         4252
nonexistent    35563
success         1373
Name: poutcome, dtype: int64

# Data Prep

1. Cleaning
2. Feature Engineering

In [9]:
df.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [10]:
cat_feature_cols = ["marital", "education", "contact", "default", "housing", "loan", "poutcome"]
num_feature_cols = ["age", "pdays", "previous", "emp.var.rate", "euribor3m", "nr.employed"]
feature_cols = cat_feature_cols + num_feature_cols 

In [11]:

X = df[feature_cols].copy()
y = df['y'].apply(lambda x: 1 if x == 'yes' else 0).copy()

# Train/Test Split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 8254)

In [13]:
X_train[cat_feature_cols]

,marital,education,contact,default,housing,loan,poutcome
31214,married,university.degree,cellular,unknown,yes,no,failure
9487,unknown,basic.6y,telephone,unknown,no,no,nonexistent
7192,married,basic.4y,telephone,no,no,no,nonexistent
12025,divorced,high.school,telephone,unknown,no,no,nonexistent
11834,married,basic.9y,telephone,no,yes,no,nonexistent
...,...,...,...,...,...,...,...
34406,married,high.school,cellular,no,no,no,nonexistent
20688,married,university.degree,cellular,no,no,yes,nonexistent
17704,single,professional.course,cellular,no,no,no,nonexistent
13574,married,basic.9y,cellular,no,no,no,nonexistent


# Feature Engineering

In [14]:
enc = OneHotEncoder(handle_unknown = 'ignore')
X_train_cat_encoded = enc.fit_transform(X_train[cat_feature_cols])
X_test_cat_encoded = enc.transform(X_test[cat_feature_cols])

In [17]:
scaler = StandardScaler()

In [18]:
X_train_num_scaled = scaler.fit_transform(X_train[num_feature_cols])
X_test_num_scaled = scaler.transform(X_test[num_feature_cols])

In [19]:
X_test_cat_encoded.shape, X_train_num_scaled.shape

((12357, 26), (28831, 6))

In [20]:
X_train = np.concatenate((X_train_cat_encoded.toarray(), X_train_num_scaled), axis = 1)
X_test = np.concatenate((X_test_cat_encoded.toarray(), X_test_num_scaled), axis=1)

In [22]:
X_test.shape, X_train.shape

((12357, 32), (28831, 32))

# Imbalanced Classes

In [27]:
sm = SMOTE()
X_balanced,y_balanced = sm.fit_resample(X_train,y_train)

# Classifier

In [28]:
rfc = RandomForestClassifier(n_estimators = 1000)

In [30]:
rfc.fit(X_balanced, y_balanced)

RandomForestClassifier(n_estimators=1000)

In [31]:
f1_score(y_test.values, rfc.predict(X_test))

0.4081041968162084

In [32]:
y_balanced.value_counts()

0    25528
1    25528
Name: y, dtype: int64

In [33]:
y_test.value_counts()

0    11020
1     1337
Name: y, dtype: int64

# Hyperparam search

In [34]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 6)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]
random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [35]:
if len(X_balanced)<1000: # can get too long!
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(
        estimator = rf, 
        param_distributions = random_grid, 
        n_iter = 10, 
        cv = 3, 
        verbose=2, 
        random_state=42, 
        n_jobs = -1,
        scoring='f1'
    )# Fit the random search model
    rf_random.fit(X_balanced, y_balanced)
    print(rf_random.best_score_)
    print(f1_score(y_test.values, rf_random.best_estimator_.predict(enc.transform(X_test))))
    print(f1_score(y_balanced.values, rf_random.best_estimator_.predict(X_balanced)))
else: 
    pass